In [69]:
import pandas as pd
import re
import numpy as np
import datetime as dt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from tqdm import tqdm

import matplotlib.pyplot as plt
%matplotlib inline

RANDOM_SEED = 322
!pip freeze > requirements.txt

#импортировали исходные данные
df = pd.read_csv('main_task.csv')
df_kaggle = pd.read_csv('kaggle_task.csv')


In [70]:
#функции обработки

#считаем % пропусков по каждому признаку
#функция missing_data(data) будет принимать на вход датасет, и определять % пропусков и тип данных
def missing_data(data):
    for i in data.columns:
        md = 100-data[i].isna().value_counts()[0]*100/len(data[i])
        print('% пропусков в',i, md)
    return

#функция конвертации строковых данных в столбце Price Range, на фход идет серия, на выходе числовые значения 
#ghbpyfrf ценовой категории
def price_cat(data):
    if data == '$':
        return 1
    elif data == '$$ - $$$':
        return 2
    elif data == '$$$$':
        return 3

#функция которая получает на вход серию со списком всех кухонь и убирает аартефакты в названиях,
#формируя список 
def resplit(ct):
    ct = ct.replace(", ",'')
    ct = ct.replace("[",'')
    ct = ct.replace("]",'')
    ct = re.split("'(.*?)'",ct)
    ct = list(filter(None, ct))
    return ct

#функция разбивки типов кухонь , нужная для выполенния заданий в модуле
def split_cuisine(ct):
    if re.findall(r"'(.*?)'", ct) == 'Unknown':
        return 'Unknown'
    else:
        return  re.findall(r"'(.*?)'", ct)

#простая функция длинны списка
def len_list(data):
    return len(data)


# функиця разделения даты из отзывов
# тип данных строка, визуально похожая на списки
# необходимо разпарсить данные
# кроме того , в строке есть дата отзыва, ее тоже надо достать
# кол-во дат будет равно кол-ву отзывов

def split_date(ct):
    if re.findall(r'\d\d/\d\d/\d{4}', ct) == 'None':
        return dt.datetime(1900,12,31)
    else:
        return  re.findall(r'\d\d/\d\d/\d{4}', ct)
    
# функция возвращает разницу между первым и последним отзывом в формате datetime
def deltadays(rd):
    if len(rd)>=2:
        a = dt.datetime.strptime(rd[0], '%m/%d/%Y') - dt.datetime.strptime(rd[len(rd)-1], '%m/%d/%Y')
        return a
    else:
        return dt.timedelta(0)

def learn_fit(df, random_state = None):
    #X - данные о ресторанах, Y - целевая переменная
    X = df.drop(['Rating', 'Restaurant_id'], axis = 1)
    y = df['Rating']
    
    # Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
    # выделим 20% данных на валидацию (параметр test_size)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=RANDOM_SEED)
    
    #создаем модель
    regr = RandomForestRegressor(n_estimators=100, verbose=0, n_jobs=-1, random_state=RANDOM_SEED)
    #обучение на тесте
    regr.fit(X_train, y_train)
    
    #используем полученную (обученную модель) для предсказания рейтинга в тестовой сборке
    #полученное значение пишем в отдульную переменную
    y_predict = regr.predict(X_test)
    
    #сравниваем предсказанное значение y_predict с тестовым y_test
    print('MAE Regression:', metrics.mean_absolute_error(y_test,y_predict))
    print('MAE Regression round state:', metrics.mean_absolute_error(y_test,np.round(y_predict*2)/2))
    
    #классификация
    le = LabelEncoder()
    le.fit(y_train)
    classif = RandomForestClassifier(n_estimators = 100, random_state=RANDOM_SEED, n_jobs=-1)
    classif.fit(X_train, le.transform(y_train))
    y_predict = classif.predict(X_test)
    print('MAE Классификация:', metrics.mean_absolute_error(y_test,le.inverse_transform(y_predict)))

In [71]:
missing_data(df)

% пропусков в Restaurant_id 0.0
% пропусков в City 0.0
% пропусков в Cuisine Style 23.207499999999996
% пропусков в Ranking 0.0
% пропусков в Rating 0.0
% пропусков в Price Range 34.715
% пропусков в Number of Reviews 6.357500000000002
% пропусков в Reviews 0.0
% пропусков в URL_TA 0.0
% пропусков в ID_TA 0.0


In [72]:
df.columns

Index(['Restaurant_id', 'City', 'Cuisine Style', 'Ranking', 'Rating',
       'Price Range', 'Number of Reviews', 'Reviews', 'URL_TA', 'ID_TA'],
      dtype='object')

In [73]:
drop_columns = ['City','Cuisine Style', 'Price Range', 'Reviews', 'URL_TA', 'ID_TA']
print('Смотрим на предикт с базовыми признаками до очистки и генерации новых\n')
learn_fit(df.drop(columns = drop_columns).dropna(), random_state=322)

Смотрим на предикт с базовыми признаками до очистки и генерации новых

MAE Regression: 0.42992353621640866
MAE Regression round state: 0.4139882541377469
MAE Классификация: 0.48723972237052854


In [74]:
#удаляем признак как не влияющий на качество 

#df.drop(columns=['ID_TA'], inplace=True)
#df_kaggle.drop(columns=['ID_TA'], inplace=True)


In [75]:
# City

df.groupby('City').Rating.agg(['mean','std','count']).sort_values(by=['mean'], ascending=False)

mean       std  count
City                                 
Rome        4.230269  0.442161   2078
Athens      4.230096  0.547579    628
Oporto      4.178363  0.594457    513
Krakow      4.146727  0.638973    443
Amsterdam   4.131215  0.567598   1086
Berlin      4.124594  0.644704   2155
Ljubljana   4.112022  0.615521    183
Budapest    4.090074  0.688316    816
Warsaw      4.089409  0.656802    727
Edinburgh   4.088087  0.672541    596
Vienna      4.078045  0.628984   1166
Dublin      4.061664  0.588444    673
Munich      4.047032  0.616500    893
Lisbon      4.046154  0.659058   1300
Zurich      4.039963  0.563295    538
Prague      4.032571  0.725964   1443
Hamburg     4.021075  0.653680    949
Copenhagen  3.985584  0.661711    659
Bratislava  3.970100  0.780451    301
Geneva      3.962578  0.658797    481
Barcelona   3.961229  0.720661   2734
London      3.955098  0.692979   5757
Lyon        3.948430  0.693183    892
Helsinki    3.946809  0.633374    376
Paris       3.942414  0.662866   4897
Luxembourg  3.914286  0.570223    210
Brussels    3.908962  0.649300   1060
Oslo        3.893506  0.643748    385
Stockholm   3.870122  0.630346    820
Milan       3.824895  0.653985   2133
Madrid      3.800837  0.737020   3108

In [76]:
df.head(5)

Restaurant_id       City                                      Cuisine Style  \
0       id_5569      Paris            ['European', 'French', 'International']   
1       id_1535  Stockholm                                                NaN   
2        id_352     London  ['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...   
3       id_3456     Berlin                                                NaN   
4        id_615     Munich  ['German', 'Central European', 'Vegetarian Fri...   

   Ranking  Rating Price Range  Number of Reviews  \
0   5570.0     3.5    $$ - $$$              194.0   
1   1537.0     4.0         NaN               10.0   
2    353.0     4.5        $$$$              688.0   
3   3458.0     5.0         NaN                3.0   
4    621.0     4.0    $$ - $$$               84.0   

                                             Reviews  \
0  [['Good food at your doorstep', 'A good hotel ...   
1  [['Unique cuisine', 'Delicious Nepalese food']...   
2  [['Catch up with friends', 'Not exceptional'],...   
3                                           [[], []]   
4  [['Best place to try a Bavarian food', 'Nice b...   

                                              URL_TA     ID_TA  
0  /Restaurant_Review-g187147-d1912643-Reviews-R_...  d1912643  
1  /Restaurant_Review-g189852-d7992032-Reviews-Bu...  d7992032  
2  /Restaurant_Review-g186338-d8632781-Reviews-RO...  d8632781  
3  /Restaurant_Review-g187323-d1358776-Reviews-Es...  d1358776  
4  /Restaurant_Review-g187309-d6864963-Reviews-Au...  d6864963

In [77]:
#импортируем данные по городам мира, и заберем оттуда только столицы (признак primary)
cities = pd.read_csv('worldcities.csv')
cities = cities[cities['capital']=='primary']

#уберем ненужную информацию и сформируем список столиц мира, хотя нам надо только европу, но все же
cities.drop(['city_ascii', 'lat','lng','capital','id','admin_name'], inplace=True, axis=1)
cities_list=list(cities['city'])

#вводим новый признак Столица, 1 если город столичный, 0 если не столичный
df['capital']=df['City'].apply(lambda x: 1 if x in cities_list else 0 )
df_kaggle['capital']=df['City'].apply(lambda x: 1 if x in cities_list else 0 )
display(df)

drop_columns = ['City','Cuisine Style', 'Price Range', 'Reviews', 'URL_TA','ID_TA']
print('Смотрим на предикт с базовыми признаками до очистки и генерации новых\n')
learn_fit(df.drop(columns = drop_columns).dropna(), random_state=322)

Restaurant_id       City  \
0           id_5569      Paris   
1           id_1535  Stockholm   
2            id_352     London   
3           id_3456     Berlin   
4            id_615     Munich   
...             ...        ...   
39995        id_499      Milan   
39996       id_6340      Paris   
39997       id_1649  Stockholm   
39998        id_640     Warsaw   
39999       id_4824      Milan   

                                           Cuisine Style  Ranking  Rating  \
0                ['European', 'French', 'International']   5570.0     3.5   
1                                                    NaN   1537.0     4.0   
2      ['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...    353.0     4.5   
3                                                    NaN   3458.0     5.0   
4      ['German', 'Central European', 'Vegetarian Fri...    621.0     4.0   
...                                                  ...      ...     ...   
39995  ['Italian', 'Vegetarian Friendly', 'Vegan Opti...    500.0     4.5   
39996  ['French', 'American', 'Bar', 'European', 'Veg...   6341.0     3.5   
39997                              ['Japanese', 'Sushi']   1652.0     4.5   
39998  ['Polish', 'European', 'Eastern European', 'Ce...    641.0     4.0   
39999                                        ['Spanish']   4827.0     3.0   

      Price Range  Number of Reviews  \
0        $$ - $$$              194.0   
1             NaN               10.0   
2            $$$$              688.0   
3             NaN                3.0   
4        $$ - $$$               84.0   
...           ...                ...   
39995    $$ - $$$               79.0   
39996    $$ - $$$              542.0   
39997         NaN                4.0   
39998    $$ - $$$               70.0   
39999    $$ - $$$              260.0   

                                                 Reviews  \
0      [['Good food at your doorstep', 'A good hotel ...   
1      [['Unique cuisine', 'Delicious Nepalese food']...   
2      [['Catch up with friends', 'Not exceptional'],...   
3                                               [[], []]   
4      [['Best place to try a Bavarian food', 'Nice b...   
...                                                  ...   
39995  [['The real Italian experience!', 'Wonderful f...   
39996  [['Parisian atmosphere', 'Bit pricey but inter...   
39997  [['Good by swedish standards', 'A hidden jewel...   
39998  [['Underground restaurant', 'Oldest Restaurant...   
39999  [['Average', 'Nice an informal'], ['01/31/2016...   

                                                  URL_TA     ID_TA  capital  
0      /Restaurant_Review-g187147-d1912643-Reviews-R_...  d1912643        1  
1      /Restaurant_Review-g189852-d7992032-Reviews-Bu...  d7992032        1  
2      /Restaurant_Review-g186338-d8632781-Reviews-RO...  d8632781        1  
3      /Restaurant_Review-g187323-d1358776-Reviews-Es...  d1358776        1  
4      /Restaurant_Review-g187309-d6864963-Reviews-Au...  d6864963        0  
...                                                  ...       ...      ...  
39995  /Restaurant_Review-g187849-d2104414-Reviews-Ro...  d2104414        0  
39996  /Restaurant_Review-g187147-d1800036-Reviews-La...  d1800036        1  
39997  /Restaurant_Review-g189852-d947615-Reviews-Sus...   d947615        1  
39998  /Restaurant_Review-g274856-d1100838-Reviews-Ho...  d1100838        1  
39999  /Restaurant_Review-g187849-d1109020-Reviews-Ta...  d1109020        0  

[40000 rows x 11 columns]

Смотрим на предикт с базовыми признаками до очистки и генерации новых

MAE Regression: 0.40157955088602443
MAE Regression round state: 0.3833956219967966
MAE Классификация: 0.44858515750133476


In [78]:
#попробуем добавить численность населения
city_list=df['City'].unique()
city_list={'Paris':2.148, 'Stockholm':0.975, 'London':8.961, 'Berlin':3.669, 'Munich':1.488, 'Oporto':0.287,
       'Milan':1.1399, 'Bratislava':0.428, 'Vienna':1.921, 'Rome':2.790, 'Barcelona':1.664, 'Madrid':3.334,
       'Dublin':1.173, 'Brussels':1.212, 'Zurich':0.434, 'Warsaw':1.793, 'Budapest':1.750, 'Copenhagen':0.794,
       'Amsterdam':0.860, 'Lyon':0.518, 'Hamburg':1.845, 'Lisbon':0.506, 'Prague':1.335, 'Oslo':0.693,
       'Helsinki':0.655, 'Edinburgh':0.488, 'Geneva':0.201, 'Ljubljana':0.286, 'Athens':0.664,
       'Luxembourg':0.644, 'Krakow':0.779}
df['population'] = df.City.replace(to_replace=city_list)
df_kaggle['population'] = df.City.replace(to_replace=city_list)


In [79]:
#проверяем модель

In [80]:
drop_columns = ['City','Cuisine Style', 'Price Range', 'Reviews', 'URL_TA','ID_TA']
print('Смотрим на предикт с базовыми признаками до очистки и генерации новых\n')
learn_fit(df.drop(columns = drop_columns).dropna(), random_state=322)

Смотрим на предикт с базовыми признаками до очистки и генерации новых

MAE Regression: 0.2298718633208756
MAE Regression round state: 0.20480512546716498
MAE Классификация: 0.2218366257341164


In [13]:
#Отлично!!! модель улучшена в 2 раза, играем дальше


In [81]:


df['Price Range'].fillna(2, inplace=True, axis=0)
range_replace={'$':1,'$$ - $$$':2,'$$$$':3}
df['Price Range']=df['Price Range'].replace(to_replace=range_replace)

#делаем тоже самое с кэгл
df_kaggle['Price Range']=df_kaggle['Price Range'].replace(to_replace=range_replace)
df_kaggle['Price Range'].fillna(2, inplace=True, axis=0)

display(df_kaggle.sample(15))
missing_data(df)
#ghjdthztv модель , уже не удаляем столбец Price Range т.к. мы его изменили 
drop_columns = ['City','Cuisine Style', 'Reviews', 'URL_TA','ID_TA']
print('Смотрим на предикт с базовыми признаками до очистки и генерации новых\n')
learn_fit(df.drop(columns = drop_columns).dropna(), random_state=322)

Restaurant_id        City  \
7038       id_7038      Madrid   
926         id_926      Madrid   
3183       id_3183   Amsterdam   
6801       id_6801      Berlin   
6795       id_6795  Luxembourg   
4173       id_4173   Amsterdam   
3996       id_3996   Stockholm   
2026       id_2026   Amsterdam   
9940       id_9940        Lyon   
8809       id_8809       Paris   
5864       id_5864      London   
2678       id_2678      London   
3159       id_3159      Krakow   
7497       id_7497      London   
6552       id_6552      Berlin   

                                          Cuisine Style  Ranking  Price Range  \
7038                                      ['Fast Food']   3810.0          2.0   
926                            ['Spanish', 'Gastropub']   4022.0          2.0   
3183  ['American', 'Dutch', 'International', 'Cafe',...   2710.0          1.0   
6801   ['American', 'Fast Food', 'Vegetarian Friendly']    233.0          1.0   
6795                                                NaN    411.0          2.0   
4173  ['Caribbean', 'Dutch', 'International', 'Europ...   2014.0          1.0   
3996                                        ['Italian']   1728.0          2.0   
2026  ['Tibetan', 'Nepali', 'Vegetarian Friendly', '...     98.0          2.0   
9940             ['American', 'Bar', 'Pub', 'Brew Pub']   1117.0          2.0   
8809                             ['French', 'European']   6391.0          2.0   
5864  ['Street Food', 'Diner', 'Vegetarian Friendly'...   1270.0          2.0   
2678                                        ['Italian']   4936.0          1.0   
3159                                                NaN    639.0          2.0   
7497        ['Cafe', 'European', 'Vegetarian Friendly']  12892.0          2.0   
6552                ['American', 'Vegetarian Friendly']     33.0          2.0   

      Number of Reviews                                            Reviews  \
7038               20.0  [["Another Irish Bar, not that it's bad :)"], ...   
926                 NaN                                           [[], []]   
3183               18.0  [['Fresh and yummy', 'Great for a Lunch snack'...   
6801              233.0  [['Tasty!', 'Great burgers, low price'], ['12/...   
6795               10.0  [['Hidden jewel!', 'Poor quality'], ['09/13/20...   
4173                6.0  [['Richtig guter laden', 'Frites!'], ['08/22/2...   
3996                2.0  [['True hole in the wall treasure'], ['07/21/2...   
2026              637.0  [['A pleasant surprise', 'Excellent Food, inti...   
9940              118.0  [['Very nice burger place!', 'Good place to ha...   
8809               19.0  [['Good quality sandwich', 'Very good quality ...   
5864              843.0  [['Late lunch visit', 'Just a cross the street...   
2678               10.0  [['Heavenly', 'Like in emilia'], ['12/20/2017'...   
3159                6.0        [['A Late Morning Brunch'], ['08/29/2017']]   
7497               20.0  [['Lovely place for lunch or tea', 'Quiet and ...   
6552              844.0  [['Far and away the BEST burgers and ribs!', '...   

                                                 URL_TA      ID_TA  capital  \
7038  /Restaurant_Review-g187514-d7599394-Reviews-Th...   d7599394        1   
926   /Restaurant_Review-g187514-d11737024-Reviews-M...  d11737024        1   
3183  /Restaurant_Review-g188590-d12146261-Reviews-T...  d12146261        0   
6801  /Restaurant_Review-g187323-d6956839-Reviews-To...   d6956839        0   
6795  /Restaurant_Review-g190356-d8490336-Reviews-Ky...   d8490336        1   
4173  /Restaurant_Review-g188590-d10462541-Reviews-B...  d10462541        1   
3996  /Restaurant_Review-g189852-d4310772-Reviews-Ca...   d4310772        1   
2026  /Restaurant_Review-g188590-d4134378-Reviews-No...   d4134378        0   
9940  /Restaurant_Review-g187265-d3607636-Reviews-Ni...   d3607636        0   
8809  /Restaurant_Review-g187147-d8050994-Reviews-Ch...   d8050994        1   
5864  /Restaurant_Review-g186338-d2693579-Rev

% пропусков в Restaurant_id 0.0
% пропусков в City 0.0
% пропусков в Cuisine Style 23.207499999999996
% пропусков в Ranking 0.0
% пропусков в Rating 0.0
% пропусков в Price Range 0.0
% пропусков в Number of Reviews 6.357500000000002
% пропусков в Reviews 0.0
% пропусков в URL_TA 0.0
% пропусков в ID_TA 0.0
% пропусков в capital 0.0
% пропусков в population 0.0
Смотрим на предикт с базовыми признаками до очистки и генерации новых

MAE Regression: 0.22897116924719696
MAE Regression round state: 0.20352375867592098
MAE Классификация: 0.22754938601174587


In [ ]:
#ничего не сильно не изменилось, кроме модели с округлением, стало даже чуть хуже
#думаю этот признак можно удалить

In [67]:
#Попробуем поиграть с кухнями

#добавили количество кухонь в ресторане
#cuisine = pd.read_csv('cuisine.csv')
#df['col_cuisine'] = cuisine['col_cuisine'].apply(resplit)
#df['col_cuisine'] = df['col_cuisine'].apply(len)

#drop_columns = ['City','Cuisine Style', 'Price Range', 'Reviews', 'URL_TA','ID_TA']
#print('Смотрим на предикт с базовыми признаками до очистки и генерации новых\n')
#learn_fit(df.drop(columns = drop_columns).dropna(), random_state=322)
#mean_cuisine=df.groupby('City').col_cuisine.agg('mean').to_dict()
#mean_cuisine['Stockholm']
#df['Cuisine Style'] = df['Cuisine Style'].map(mean_cuisine)
#df['Cuisine Style'] = df['City'].replace(to_replace=mean_cuisine)
#df_kaggle['Cuisine Style'] = df['City'].replace(to_replace=mean_cuisine)


#df.sample(15)
#missing_data(df)

#drop_columns = ['City','Number of Reviews', 'URL_TA', 'ID_TA', 'Reviews']
#print('Смотрим на предикт с базовыми признаками до очистки и генерации новых\n')
#learn_fit(df.drop(columns = drop_columns).dropna(), random_state=322)

#модель ухудшилась, убираем этот признак !!! 

% пропусков в Restaurant_id 0.0
% пропусков в City 0.0
% пропусков в Cuisine Style 0.0
% пропусков в Ranking 0.0
% пропусков в Rating 0.0
% пропусков в Price Range 0.0
% пропусков в Number of Reviews 6.357500000000002
% пропусков в Reviews 0.0
% пропусков в URL_TA 0.0
% пропусков в ID_TA 0.0
% пропусков в capital 0.0
% пропусков в population 0.0
% пропусков в col_cuisine 0.0
Смотрим на предикт с базовыми признаками до очистки и генерации новых

MAE Regression: 0.3123705
MAE Regression round state: 0.29555
MAE Классификация: 0.35065


In [68]:
df.head(4)

Restaurant_id       City  Cuisine Style  Ranking  Rating  Price Range  \
0       id_5569      Paris       2.148458   5570.0     3.5            2   
1       id_1535  Stockholm       2.446341   1537.0     4.0            2   
2        id_352     London       2.932604    353.0     4.5            3   
3       id_3456     Berlin       2.271462   3458.0     5.0            2   

   Number of Reviews                                            Reviews  \
0              194.0  [['Good food at your doorstep', 'A good hotel ...   
1               10.0  [['Unique cuisine', 'Delicious Nepalese food']...   
2              688.0  [['Catch up with friends', 'Not exceptional'],...   
3                3.0                                           [[], []]   

                                              URL_TA     ID_TA  capital  \
0  /Restaurant_Review-g187147-d1912643-Reviews-R_...  d1912643        1   
1  /Restaurant_Review-g189852-d7992032-Reviews-Bu...  d7992032        1   
2  /Restaurant_Review-g186338-d8632781-Reviews-RO...  d8632781        1   
3  /Restaurant_Review-g187323-d1358776-Reviews-Es...  d1358776        1   

   population  col_cuisine  
0       2.148            3  
1       0.975            2  
2       8.961            7  
3       3.669            0

In [254]:
#есть небольшое улучшение - изменим и тестовую

#df

#df3=df[['URL_TA','col_cuisine']]

#df4 = pd.merge(df_kaggle, df3, how = 'left')
#display(df4)
#missing_data(df4)
#display(df[df['Restaurant_id']=='id_0'])
#display(df_kaggle[df_kaggle['Restaurant_id']=='id_0'])
#df_kaggle['col_cuisine'] = cuisine['col_cuisine'].apply(resplit)
#df_kaggle['col_cuisine'] = df['col_cuisine'].apply(len)

In [82]:
#создадим признак, кол-во ресторанов в городе
per_city_dict = dict(df['City'].value_counts())
per_city_dict
df['city_restaurants'] = df.City.map(per_city_dict)

# Добавим признак - нормализованный ранг ranking_norm
df['ranking_norm'] = df.Ranking / df.city_restaurants

drop_columns = ['City','Cuisine Style', 'Price Range', 'Reviews', 'URL_TA','ID_TA']
print('Смотрим на предикт с базовыми признаками до очистки и генерации новых\n')
learn_fit(df.drop(columns = drop_columns).dropna(), random_state=322)
df.head(3)

Смотрим на предикт с базовыми признаками до очистки и генерации новых

MAE Regression: 0.22922744260544584
MAE Regression round state: 0.20197544046983448
MAE Классификация: 0.21238654564869194


Restaurant_id       City                                      Cuisine Style  \
0       id_5569      Paris            ['European', 'French', 'International']   
1       id_1535  Stockholm                                                NaN   
2        id_352     London  ['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...   

   Ranking  Rating  Price Range  Number of Reviews  \
0   5570.0     3.5            2              194.0   
1   1537.0     4.0            2               10.0   
2    353.0     4.5            3              688.0   

                                             Reviews  \
0  [['Good food at your doorstep', 'A good hotel ...   
1  [['Unique cuisine', 'Delicious Nepalese food']...   
2  [['Catch up with friends', 'Not exceptional'],...   

                                              URL_TA     ID_TA  capital  \
0  /Restaurant_Review-g187147-d1912643-Reviews-R_...  d1912643        1   
1  /Restaurant_Review-g189852-d7992032-Reviews-Bu...  d7992032        1   
2  /Restaurant_Review-g186338-d8632781-Reviews-RO...  d8632781        1   

   population  city_restaurants  ranking_norm  
0       2.148              4897      1.137431  
1       0.975               820      1.874390  
2       8.961              5757      0.061317

In [86]:
#отличный результат , повысили еще точность
#сделаем тоже и на kaggle

per_city_dict = dict(df_kaggle['City'].value_counts())
per_city_dict
df_kaggle['city_restaurants'] = df_kaggle.City.map(per_city_dict)
# Добавим признак - нормализованный ранг ranking_norm
df_kaggle['ranking_norm'] = df_kaggle.Ranking / df_kaggle.city_restaurants
df_kaggle.head(3)

Restaurant_id       City                                      Cuisine Style  \
0          id_0      Paris                                     ['Bar', 'Pub']   
1          id_1   Helsinki  ['European', 'Scandinavian', 'Gluten Free Opti...   
2          id_2  Edinburgh                            ['Vegetarian Friendly']   

   Ranking  Price Range  Number of Reviews  \
0  12963.0          2.0                4.0   
1    106.0          2.0               97.0   
2    810.0          2.0               28.0   

                                             Reviews  \
0                                           [[], []]   
1  [['Very good reviews!', 'Fine dining in Hakani...   
2  [['Better than the Links', 'Ivy Black'], ['12/...   

                                              URL_TA      ID_TA  capital  \
0  /Restaurant_Review-g187147-d10746918-Reviews-L...  d10746918        1   
1  /Restaurant_Review-g189934-d6674944-Reviews-Ra...   d6674944        1   
2  /Restaurant_Review-g186525-d13129638-Reviews-B...  d13129638        1   

   population  city_restaurants  ranking_norm  
0       2.148              1211     10.704377  
1       0.975                99      1.070707  
2       8.961               149      5.436242

In [387]:
df.head(3)

Restaurant_id                                      Cuisine Style  Ranking  \
0       id_5569            ['European', 'French', 'International']   5570.0   
1       id_1535                                                NaN   1537.0   
2        id_352  ['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...    353.0   

   Rating  Price Range  Number of Reviews  \
0     3.5            2              194.0   
1     4.0            2               10.0   
2     4.5            3              688.0   

                                             Reviews  \
0  [['Good food at your doorstep', 'A good hotel ...   
1  [['Unique cuisine', 'Delicious Nepalese food']...   
2  [['Catch up with friends', 'Not exceptional'],...   

                                              URL_TA     ID_TA  capital  ...  \
0  /Restaurant_Review-g187147-d1912643-Reviews-R_...  d1912643        1  ...   
1  /Restaurant_Review-g189852-d7992032-Reviews-Bu...  d7992032        1  ...   
2  /Restaurant_Review-g186338-d8632781-Reviews-RO...  d8632781        1  ...   

   City_Oporto  City_Oslo  City_Paris  City_Prague  City_Rome  City_Stockholm  \
0            0          0           1            0          0               0   
1            0          0           0            0          0               1   
2            0          0           0            0          0               0   

   City_Vienna  City_Warsaw  City_Zurich  City_nan  
0            0            0            0         0  
1            0            0            0         0  
2            0            0            0         0  

[3 rows x 46 columns]

In [385]:
#делаем dummy
#df = pd.get_dummies(df, columns=[ 'City',], dummy_na=True)
df_kaggle = pd.get_dummies(df_kaggle, columns=[ 'City',], dummy_na=True)
df_kaggle.head(3)


Restaurant_id                                      Cuisine Style  Ranking  \
0          id_0                                     ['Bar', 'Pub']  12963.0   
1          id_1  ['European', 'Scandinavian', 'Gluten Free Opti...    106.0   
2          id_2                            ['Vegetarian Friendly']    810.0   

  Price Range  Number of Reviews  \
0    $$ - $$$                4.0   
1    $$ - $$$               97.0   
2    $$ - $$$               28.0   

                                             Reviews  \
0                                           [[], []]   
1  [['Very good reviews!', 'Fine dining in Hakani...   
2  [['Better than the Links', 'Ivy Black'], ['12/...   

                                              URL_TA      ID_TA  capital  \
0  /Restaurant_Review-g187147-d10746918-Reviews-L...  d10746918        1   
1  /Restaurant_Review-g189934-d6674944-Reviews-Ra...   d6674944        1   
2  /Restaurant_Review-g186525-d13129638-Reviews-B...  d13129638        1   

   population  ...  City_Oporto  City_Oslo  City_Paris  City_Prague  \
0       2.148  ...            0          0           1            0   
1       0.975  ...            0          0           0            0   
2       8.961  ...            0          0           0            0   

   City_Rome  City_Stockholm  City_Vienna  City_Warsaw  City_Zurich  City_nan  
0          0               0            0            0            0         0  
1          0               0            0            0            0         0  
2          0               0            0            0            0         0  

[3 rows x 44 columns]

In [87]:
drop_columns = [ 'Cuisine Style', 'Price Range', 'Reviews', 'URL_TA','ID_TA']
print('Смотрим на предикт с базовыми признаками до очистки и генерации новых\n')
learn_fit(df.drop(columns = drop_columns).dropna(), random_state=322)

Смотрим на предикт с базовыми признаками до очистки и генерации новых



ValueError: could not convert string to float: 'Lyon'

In [ ]:
#еще чуть чуть улучшили прогноз

In [388]:
#смотрим кухни

In [391]:
# здесь не учитываются пропуски, надо их убрать раньше либо сейас, заменив на данные парсинга


df['Cuisine Style']=df['Cuisine Style'].str.findall(r"'([\w\s]*)'")
df_kaggle

Restaurant_id                                      Cuisine Style  \
0             id_0                                     ['Bar', 'Pub']   
1             id_1  ['European', 'Scandinavian', 'Gluten Free Opti...   
2             id_2                            ['Vegetarian Friendly']   
3             id_3  ['Italian', 'Mediterranean', 'European', 'Vege...   
4             id_4  ['Italian', 'Mediterranean', 'European', 'Seaf...   
...            ...                                                ...   
9995       id_9995  ['Caribbean', 'Barbecue', 'Jamaican', 'Vegetar...   
9996       id_9996       ['Indian', 'Asian', 'Nepali', 'Bangladeshi']   
9997       id_9997                                                NaN   
9998       id_9998                                        ['Italian']   
9999       id_9999                                                NaN   

      Ranking Price Range  Number of Reviews  \
0     12963.0    $$ - $$$                4.0   
1       106.0    $$ - $$$               97.0   
2       810.0    $$ - $$$               28.0   
3      1669.0        $$$$              202.0   
4        37.0        $$$$              162.0   
...       ...         ...                ...   
9995   4367.0           $               28.0   
9996   2612.0    $$ - $$$                2.0   
9997   5562.0         NaN                2.0   
9998   4010.0         NaN                2.0   
9999   6791.0         NaN               14.0   

                                                Reviews  \
0                                              [[], []]   
1     [['Very good reviews!', 'Fine dining in Hakani...   
2     [['Better than the Links', 'Ivy Black'], ['12/...   
3     [['Most exquisite', 'Delicious and authentic']...   
4     [['Always the best in bratislava', 'Very good ...   
...                                                 ...   
9995  [['By far best Chicken burgers and wings in L....   
9996               [['Ok indian food'], ['11/25/2017']]   
9997                                           [[], []]   
9998                                           [[], []]   
9999  [['Ethiopian Dining in Pleasant Space'], ['07/...   

                                                 URL_TA      ID_TA  capital  \
0     /Restaurant_Review-g187147-d10746918-Reviews-L...  d10746918        1   
1     /Restaurant_Review-g189934-d6674944-Reviews-Ra...   d6674944        1   
2     /Restaurant_Review-g186525-d13129638-Reviews-B...  d13129638        1   
3     /Restaurant_Review-g186338-d680417-Reviews-Qui...    d680417        1   
4     /Restaurant_Review-g274924-d1112354-Reviews-Ma...   d1112354        0   
...                                                 ...        ...      ...   
9995  /Restaurant_Review-g186338-d11916827-Reviews-C...  d11916827        1   
9996  /Restaurant_Review-g189158-d12701633-Reviews-C...  d12701633        0   
9997  /Restaurant_Review-g187849-d13198194-Reviews-Q...  d13198194        1   
9998  /Restaurant_Review-g187849-d3862815-Reviews-La...   d3862815        1   
9999  /Restaurant_Review-g187147-d10342814-Reviews-L...  d10342814        1   

      population  ...  City_Oporto  City_Oslo  City_Paris  City_Prague  \
0          2.148  ...            0          0           1            0   
1          0.975  ...            0          0           0            0   
2          8.961  ...            0          0           0            0   
3          3.669  ...            0          0           0            0   
4          1.488  ...            0          0           0            0   
...          ...  ...          ...        ...         ...          ...   
9995       8.961  ...            0          0           0            0   
9996       0.488  ...            0          0           0            0   
9997       0.506  ...            0          0           0            0   
9998       0.428  ...            0          0           0            0   
9999       2.790  ...            0          0           1            0   

    